# 🔒 Test de Stéganographie JPEG
## Service de Dissimulation et Extraction de Messages dans les Images JPEG

Ce notebook teste toutes les fonctionnalités du service de stéganographie JPEG développé pour l'application Stegano-Flask.

### Fonctionnalités testées :
- 📊 **Analyse de capacité** des images JPEG
- 🔤 **Dissimulation EXIF** : Messages dans les métadonnées
- 🔢 **Dissimulation LSB** : Modification des bits de poids faible  
- 📝 **Extraction de messages** cachés
- ✍️ **Signature stéganographique** pour l'intégrité
- ✅ **Vérification de signature** et détection de modifications
- ⚡ **Comparaison des performances** entre méthodes

---

In [1]:
# Import des bibliothèques requises
import sys
import os
sys.path.append('../')  # Ajouter le répertoire parent au chemin

# Bibliothèques de base
import numpy as np
import hashlib
import json
import time
from datetime import datetime
from pathlib import Path

# Bibliothèques d'image
from PIL import Image, ExifTags
from PIL.ExifTags import TAGS
import piexif

# Import du service JPEG stéganographie
try:
    from app.services.jpeg_steganography_service import JPEGSteganographyService
    print("✅ Service JPEG Steganography importé avec succès")
except ImportError as e:
    print(f"❌ Erreur d'import: {e}")
    print("Vérifiez que le service existe dans app/services/")

# Bibliothèques pour l'affichage
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from IPython.display import display, HTML, Markdown

print("📚 Toutes les bibliothèques importées avec succès !")
print(f"🕐 Notebook initialisé le : {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


✅ Service JPEG Steganography importé avec succès
✅ Service JPEG Steganography importé avec succès
📚 Toutes les bibliothèques importées avec succès !
🕐 Notebook initialisé le : 2025-07-17 22:47:49
📚 Toutes les bibliothèques importées avec succès !
🕐 Notebook initialisé le : 2025-07-17 22:47:49


In [2]:
# Configuration et initialisation du service
print("🔧 Configuration du service de stéganographie JPEG...\n")

# Créer les dossiers nécessaires
test_dir = Path("../test_images")
output_dir = Path("../notebooks/test_outputs")
output_dir.mkdir(exist_ok=True)

print(f"📁 Dossier de test : {test_dir}")
print(f"📁 Dossier de sortie : {output_dir}")

# Initialiser le service JPEG
try:
    jpeg_service = JPEGSteganographyService()
    print("✅ Service JPEG Steganography initialisé")
except Exception as e:
    print(f"❌ Erreur d'initialisation du service : {e}")

# Fonction utilitaire pour afficher les informations d'image
def display_image_info(image_path):
    """Affiche les informations détaillées d'une image."""
    if not os.path.exists(image_path):
        print(f"❌ Fichier non trouvé : {image_path}")
        return

    try:
        with Image.open(image_path) as img:
            print(f"📷 Image : {os.path.basename(image_path)}")
            print(f"   • Format : {img.format}")
            print(f"   • Taille : {img.size[0]} x {img.size[1]} pixels")
            print(f"   • Mode : {img.mode}")

            # Taille du fichier
            size_bytes = os.path.getsize(image_path)
            size_kb = size_bytes / 1024
            print(f"   • Taille fichier : {size_kb:.1f} KB ({size_bytes:,} bytes)")

            # Vérifier les métadonnées EXIF
            exif_dict = piexif.load(image_path)
            exif_count = sum(len(ifd) for ifd in exif_dict.values() if isinstance(ifd, dict))
            print(f"   • Métadonnées EXIF : {exif_count} entrées")

    except Exception as e:
        print(f"❌ Erreur lors de l'analyse : {e}")

print("\n🛠️ Configuration terminée !")
print("=" * 50)

🔧 Configuration du service de stéganographie JPEG...

📁 Dossier de test : ..\test_images
📁 Dossier de sortie : ..\notebooks\test_outputs
✅ Service JPEG Steganography initialisé

🛠️ Configuration terminée !


## 📊 1. Analyse de la Capacité d'Images JPEG

Cette section teste la fonction `analyze_jpeg_capacity()` qui évalue combien de données peuvent être cachées dans une image JPEG selon différentes méthodes.

In [3]:
# Test d'analyse de capacité sur plusieurs images
print("🔍 Analyse de la capacité de dissimulation...\n")

# Liste des images de test disponibles
test_images = [
    "../test_images/test_image_1.jpg",
    "../test_images/test_image_3.jpg",
    "../test_images/test_image_1_steg.jpg"
]

capacity_results = []

for img_path in test_images:
    if os.path.exists(img_path):
        print(f"📸 Analyse de : {os.path.basename(img_path)}")
        print("-" * 40)

        # Afficher les informations de base
        display_image_info(img_path)

        try:
            # Analyser la capacité
            capacity = jpeg_service.analyze_jpeg_capacity(img_path)

            if 'error' not in capacity:
                print(f"\n💾 Capacités de dissimulation :")
                print(f"   • EXIF : {capacity.get('exif_capacity', 0):,} caractères")
                print(f"   • LSB : {capacity.get('lsb_capacity', 0):,} caractères")

                # Calculer les pourcentages
                file_size = os.path.getsize(img_path)
                exif_percent = (capacity.get('exif_capacity', 0) / file_size) * 100
                lsb_percent = (capacity.get('lsb_capacity', 0) / file_size) * 100

                print(f"   • EXIF (% du fichier) : {exif_percent:.2f}%")
                print(f"   • LSB (% du fichier) : {lsb_percent:.2f}%")

                capacity_results.append({
                    'image': os.path.basename(img_path),
                    'file_size': file_size,
                    'exif_capacity': capacity.get('exif_capacity', 0),
                    'lsb_capacity': capacity.get('lsb_capacity', 0),
                    'exif_percent': exif_percent,
                    'lsb_percent': lsb_percent
                })

            else:
                print(f"❌ Erreur : {capacity['error']}")

        except Exception as e:
            print(f"❌ Erreur lors de l'analyse : {e}")

        print("\n" + "=" * 50 + "\n")
    else:
        print(f"⚠️ Image non trouvée : {img_path}")

print(f"✅ Analyse terminée pour {len(capacity_results)} images")

# Résumé des résultats
if capacity_results:
    print("\n📈 Résumé des capacités :")
    for result in capacity_results:
        print(f"• {result['image']} :")
        print(f"  EXIF: {result['exif_capacity']:,} chars ({result['exif_percent']:.1f}%)")
        print(f"  LSB:  {result['lsb_capacity']:,} chars ({result['lsb_percent']:.1f}%)")

🔍 Analyse de la capacité de dissimulation...

📸 Analyse de : test_image_1.jpg
----------------------------------------
📷 Image : test_image_1.jpg
   • Format : JPEG
   • Taille : 194 x 259 pixels
   • Mode : RGB
   • Taille fichier : 9.9 KB (10,144 bytes)
   • Métadonnées EXIF : 0 entrées

💾 Capacités de dissimulation :
   • EXIF : 0 caractères
   • LSB : 0 caractères
   • EXIF (% du fichier) : 0.00%
   • LSB (% du fichier) : 0.00%


📸 Analyse de : test_image_3.jpg
----------------------------------------
📷 Image : test_image_3.jpg
   • Format : JPEG
   • Taille : 1400 x 933 pixels
   • Mode : RGB
   • Taille fichier : 275.8 KB (282,373 bytes)
   • Métadonnées EXIF : 0 entrées

💾 Capacités de dissimulation :
   • EXIF : 0 caractères
   • LSB : 0 caractères
   • EXIF (% du fichier) : 0.00%
   • LSB (% du fichier) : 0.00%


📸 Analyse de : test_image_1_steg.jpg
----------------------------------------
📷 Image : test_image_1_steg.jpg
   • Format : JPEG
   • Taille : 194 x 259 pixels
   • M

## 🔤 2. Dissimulation avec la Méthode EXIF

Test de la dissimulation de messages dans les métadonnées EXIF. Cette méthode est idéale pour les petits messages car elle n'affecte pas la qualité visuelle de l'image.

In [4]:
# Test de dissimulation EXIF
print("🔐 Test de dissimulation avec méthode EXIF...\n")

# Message de test
test_message = "Ceci est un message secret caché dans les métadonnées EXIF de cette image JPEG ! 🔒"
print(f"📝 Message à cacher : \"{test_message}\"")
print(f"📏 Longueur du message : {len(test_message)} caractères\n")

# Image source
source_image = "../test_images/test_image_1.jpg"
if not os.path.exists(source_image):
    print(f"❌ Image source non trouvée : {source_image}")
else:
    print(f"📷 Image source : {os.path.basename(source_image)}")
    display_image_info(source_image)

    # Fichier de sortie
    output_file = output_dir / "test_exif_hidden.jpg"

    try:
        # Cacher le message avec la méthode EXIF
        print(f"\n🔧 Dissimulation en cours...")
        result = jpeg_service.hide_message_in_jpeg(
            str(source_image),
            test_message,
            str(output_file),
            method='exif'
        )

        if result['success']:
            print(f"✅ Message caché avec succès !")
            print(f"💾 Fichier créé : {output_file.name}")
            print(f"📊 Détails :")
            print(f"   • Méthode : {result.get('method', 'exif')}")
            print(f"   • Longueur message : {result.get('message_length', 0)} caractères")

            # Comparer les tailles de fichier
            original_size = os.path.getsize(source_image)
            new_size = os.path.getsize(output_file)
            size_diff = new_size - original_size

            print(f"   • Taille originale : {original_size:,} bytes")
            print(f"   • Nouvelle taille : {new_size:,} bytes")
            print(f"   • Différence : {size_diff:+,} bytes ({(size_diff/original_size)*100:+.2f}%)")

            # Afficher les informations du fichier créé
            print(f"\n📋 Informations du fichier avec message caché :")
            display_image_info(str(output_file))

        else:
            print(f"❌ Échec de la dissimulation : {result.get('error', 'Erreur inconnue')}")

    except Exception as e:
        print(f"❌ Erreur lors de la dissimulation : {e}")

print("\n" + "=" * 60)

Erreur lors de la dissimulation: Erreur EXIF: a bytes-like object is required, not 'dict'


🔐 Test de dissimulation avec méthode EXIF...

📝 Message à cacher : "Ceci est un message secret caché dans les métadonnées EXIF de cette image JPEG ! 🔒"
📏 Longueur du message : 82 caractères

📷 Image source : test_image_1.jpg
📷 Image : test_image_1.jpg
   • Format : JPEG
   • Taille : 194 x 259 pixels
   • Mode : RGB
   • Taille fichier : 9.9 KB (10,144 bytes)
   • Métadonnées EXIF : 0 entrées

🔧 Dissimulation en cours...
❌ Échec de la dissimulation : Erreur EXIF: a bytes-like object is required, not 'dict'



## 🔢 3. Dissimulation avec la Méthode LSB

Test de la méthode LSB (Least Significant Bit) qui modifie les bits de poids faible des pixels. Cette méthode offre une plus grande capacité que EXIF mais peut légèrement affecter la qualité de l'image.

In [5]:
# Test de dissimulation LSB
print("🔢 Test de dissimulation avec méthode LSB...\n")

# Message de test pour LSB (peut être plus long)
lsb_message = """Ceci est un message plus long pour tester la méthode LSB (Least Significant Bit).
Cette méthode permet de cacher des messages plus longs en modifiant les bits de poids faible des pixels.
Elle offre une grande capacité mais peut légèrement affecter la qualité de l'image.
Le message peut contenir des caractères spéciaux : àéèêëïôöùûüÿç !@#$%^&*()
Et même des emojis : 🔒🔑📷💾🎯✨🚀"""

print(f"📝 Message à cacher : \"{lsb_message[:80]}...\"")
print(f"📏 Longueur du message : {len(lsb_message)} caractères\n")

# Image source
source_image = "../test_images/test_image_3.jpg"
if not os.path.exists(source_image):
    print(f"❌ Image source non trouvée : {source_image}")
else:
    print(f"📷 Image source : {os.path.basename(source_image)}")
    display_image_info(source_image)

    # Vérifier la capacité avant de cacher
    try:
        capacity = jpeg_service.analyze_jpeg_capacity(source_image)
        lsb_capacity = capacity.get('lsb_capacity', 0)
        print(f"\n💾 Capacité LSB disponible : {lsb_capacity:,} caractères")

        if len(lsb_message) > lsb_capacity:
            print(f"⚠️ Message trop long ! Réduction à {lsb_capacity} caractères...")
            lsb_message = lsb_message[:lsb_capacity]
        else:
            print(f"✅ Message compatible avec la capacité")
    except Exception as e:
        print(f"⚠️ Impossible de vérifier la capacité : {e}")

    # Fichier de sortie
    output_file = output_dir / "test_lsb_hidden.jpg"

    try:
        # Cacher le message avec la méthode LSB
        print(f"\n🔧 Dissimulation LSB en cours...")
        start_time = time.time()

        result = jpeg_service.hide_message_in_jpeg(
            str(source_image),
            lsb_message,
            str(output_file),
            method='lsb'
        )

        end_time = time.time()
        processing_time = end_time - start_time

        if result['success']:
            print(f"✅ Message caché avec succès !")
            print(f"💾 Fichier créé : {output_file.name}")
            print(f"⏱️ Temps de traitement : {processing_time:.2f} secondes")
            print(f"📊 Détails :")
            print(f"   • Méthode : {result.get('method', 'lsb')}")
            print(f"   • Longueur message : {result.get('message_length', 0)} caractères")

            # Comparer les tailles de fichier
            original_size = os.path.getsize(source_image)
            new_size = os.path.getsize(output_file)
            size_diff = new_size - original_size

            print(f"   • Taille originale : {original_size:,} bytes")
            print(f"   • Nouvelle taille : {new_size:,} bytes")
            print(f"   • Différence : {size_diff:+,} bytes ({(size_diff/original_size)*100:+.2f}%)")

            # Calculer les statistiques de performance
            chars_per_second = len(lsb_message) / processing_time if processing_time > 0 else 0
            print(f"   • Vitesse : {chars_per_second:.0f} caractères/seconde")

            # Afficher les informations du fichier créé
            print(f"\n📋 Informations du fichier avec message caché :")
            display_image_info(str(output_file))

        else:
            print(f"❌ Échec de la dissimulation : {result.get('error', 'Erreur inconnue')}")

    except Exception as e:
        print(f"❌ Erreur lors de la dissimulation LSB : {e}")

print("\n" + "=" * 60)

🔢 Test de dissimulation avec méthode LSB...

📝 Message à cacher : "Ceci est un message plus long pour tester la méthode LSB (Least Significant Bit)..."
📏 Longueur du message : 375 caractères

📷 Image source : test_image_3.jpg


📷 Image : test_image_3.jpg
   • Format : JPEG
   • Taille : 1400 x 933 pixels
   • Mode : RGB
   • Taille fichier : 275.8 KB (282,373 bytes)
   • Métadonnées EXIF : 0 entrées

💾 Capacité LSB disponible : 0 caractères
⚠️ Message trop long ! Réduction à 0 caractères...

🔧 Dissimulation LSB en cours...
✅ Message caché avec succès !
💾 Fichier créé : test_lsb_hidden.jpg
⏱️ Temps de traitement : 0.18 secondes
📊 Détails :
   • Méthode : lsb
   • Longueur message : 0 caractères
✅ Message caché avec succès !
💾 Fichier créé : test_lsb_hidden.jpg
⏱️ Temps de traitement : 0.18 secondes
📊 Détails :
   • Méthode : lsb
   • Longueur message : 0 caractères
   • Taille originale : 282,373 bytes
   • Nouvelle taille : 467,265 bytes
   • Différence : +184,892 bytes (+65.48%)
   • Vitesse : 0 caractères/seconde

📋 Informations du fichier avec message caché :
📷 Image : test_lsb_hidden.jpg
   • Format : JPEG
   • Taille : 1400 x 933 pixels
   • Mode : RGB
   • Taille fichier : 456.3 KB (467,265 bytes)
   • 

## 📝 4. Extraction de Messages Cachés

Cette section teste l'extraction des messages cachés dans les images créées précédemment. Nous testons l'extraction pour les deux méthodes : EXIF et LSB.

In [6]:
# Vérification et création des fichiers de test si nécessaire
print("🔍 VÉRIFICATION DES FICHIERS DE TEST")
print("=" * 45)

# Vérifier si les fichiers de test existent
test_files_needed = [
    {'file': output_dir / "test_exif_hidden.jpg", 'method': 'exif', 'type': 'EXIF'},
    {'file': output_dir / "test_lsb_hidden.jpg", 'method': 'lsb', 'type': 'LSB'}
]

files_to_create = []
for test_file in test_files_needed:
    if test_file['file'].exists():
        print(f"✅ Fichier {test_file['type']} trouvé : {test_file['file'].name}")
    else:
        print(f"❌ Fichier {test_file['type']} manquant : {test_file['file'].name}")
        files_to_create.append(test_file)

if files_to_create:
    print(f"\n🔧 CRÉATION DES FICHIERS DE TEST MANQUANTS")
    print("-" * 40)

    # Image source pour les tests
    source_image = "../test_images/test_image_1.jpg"

    if not os.path.exists(source_image):
        # Essayer d'autres images disponibles
        alternative_images = [
            "../test_images/test_image_3.jpg",
            "../test_images/test_image_1_steg.jpg",
            "../test_images/test_image_2.png",
            "../test_images/test_image_4.png"
        ]

        for alt_img in alternative_images:
            if os.path.exists(alt_img):
                source_image = alt_img
                print(f"📷 Utilisation de l'image alternative : {os.path.basename(source_image)}")
                break
        else:
            print("❌ Aucune image de test disponible !")
            print("💡 Veuillez placer une image JPEG dans le dossier test_images/")
    else:
        print(f"📷 Utilisation de l'image source : {os.path.basename(source_image)}")

    if os.path.exists(source_image):
        # Messages de test
        test_messages = {
            'exif': "Ceci est un message secret caché dans les métadonnées EXIF de cette image JPEG ! 🔒",
            'lsb': """Ceci est un message plus long pour tester la méthode LSB (Least Significant Bit).
Cette méthode permet de cacher des messages plus longs en modifiant les bits de poids faible des pixels.
Elle offre une grande capacité mais peut légèrement affecter la qualité de l'image.
Le message peut contenir des caractères spéciaux : àéèêëïôöùûüÿç !@#$%^&*()
Et même des emojis : 🔒🔑📷💾🎯✨🚀"""
        }

        # Créer les fichiers manquants
        for test_file in files_to_create:
            method = test_file['method']
            output_file = test_file['file']
            message = test_messages[method]

            print(f"\n🔧 Création du fichier {test_file['type']}...")
            print(f"   📝 Message : \"{message[:50]}...\"")
            print(f"   📏 Longueur : {len(message)} caractères")

            try:
                result = jpeg_service.hide_message_in_jpeg(
                    source_image,
                    message,
                    str(output_file),
                    method=method
                )

                if result['success']:
                    print(f"   ✅ Fichier {test_file['type']} créé avec succès !")
                    print(f"   💾 Fichier : {output_file.name}")

                    # Mettre à jour les variables globales pour les tests d'extraction
                    if method == 'exif':
                        globals()['test_message'] = message
                    elif method == 'lsb':
                        globals()['lsb_message'] = message

                else:
                    print(f"   ❌ Échec de création : {result.get('error', 'Erreur inconnue')}")

            except Exception as e:
                print(f"   ❌ Erreur lors de la création : {e}")

        print(f"\n✅ Vérification terminée !")
    else:
        print("❌ Impossible de créer les fichiers de test sans image source")

else:
    print(f"\n✅ Tous les fichiers de test sont présents !")

print("=" * 45)

Erreur lors de la dissimulation: Erreur EXIF: a bytes-like object is required, not 'dict'


🔍 VÉRIFICATION DES FICHIERS DE TEST
❌ Fichier EXIF manquant : test_exif_hidden.jpg
✅ Fichier LSB trouvé : test_lsb_hidden.jpg

🔧 CRÉATION DES FICHIERS DE TEST MANQUANTS
----------------------------------------
📷 Utilisation de l'image source : test_image_1.jpg

🔧 Création du fichier EXIF...
   📝 Message : "Ceci est un message secret caché dans les métadonn..."
   📏 Longueur : 82 caractères
   ❌ Échec de création : Erreur EXIF: a bytes-like object is required, not 'dict'

✅ Vérification terminée !


In [ ]:
# Test d'extraction de messages
print("🔍 Test d'extraction de messages cachés...\n")

# Liste des fichiers à tester pour l'extraction
test_files = [
    {
        'file': output_dir / "test_exif_hidden.jpg",
        'method': 'exif',
        'expected_message': test_message,
        'description': 'Message EXIF'
    },
    {
        'file': output_dir / "test_lsb_hidden.jpg",
        'method': 'lsb',
        'expected_message': lsb_message,
        'description': 'Message LSB'
    }
]

extraction_results = []

for test_case in test_files:
    file_path = test_case['file']
    method = test_case['method']
    expected = test_case['expected_message']
    description = test_case['description']

    print(f"🔎 Test d'extraction : {description}")
    print(f"📁 Fichier : {file_path.name}")
    print(f"🔧 Méthode : {method.upper()}")
    print("-" * 50)

    if not file_path.exists():
        print(f"❌ Fichier non trouvé : {file_path}")
        print()
        continue

    try:
        # Extraire le message
        start_time = time.time()
        result = jpeg_service.extract_message_from_jpeg(str(file_path), method=method)
        end_time = time.time()
        extraction_time = end_time - start_time

        if result['success']:
            extracted_message = result['message']

            print(f"✅ Extraction réussie !")
            print(f"⏱️ Temps d'extraction : {extraction_time:.3f} secondes")
            print(f"📏 Longueur extraite : {len(extracted_message)} caractères")

            # Vérifier si le message correspond
            if extracted_message == expected:
                print(f"✅ Message parfaitement identique !")
                match_status = "✅ PARFAIT"
            elif extracted_message.strip() == expected.strip():
                print(f"✅ Message identique (espaces ignorés)")
                match_status = "✅ IDENTIQUE"
            else:
                print(f"⚠️ Message différent de l'original")
                match_status = "⚠️ DIFFÉRENT"

                # Analyser les différences
                if len(extracted_message) != len(expected):
                    print(f"   • Longueur attendue : {len(expected)}")
                    print(f"   • Longueur extraite : {len(extracted_message)}")

                # Afficher un aperçu des messages
                print(f"   • Attendu : \"{expected[:50]}...\"")
                print(f"   • Extrait : \"{extracted_message[:50]}...\"")

            # Afficher un aperçu du message extrait
            if len(extracted_message) <= 100:
                print(f"💬 Message extrait : \"{extracted_message}\"")
            else:
                print(f"💬 Aperçu du message : \"{extracted_message[:100]}...\"")

            extraction_results.append({
                'method': method,
                'file': file_path.name,
                'success': True,
                'extraction_time': extraction_time,
                'message_length': len(extracted_message),
                'match_status': match_status
            })

        else:
            print(f"❌ Échec de l'extraction : {result.get('error', 'Erreur inconnue')}")
            extraction_results.append({
                'method': method,
                'file': file_path.name,
                'success': False,
                'error': result.get('error', 'Erreur inconnue')
            })

    except Exception as e:
        print(f"❌ Erreur lors de l'extraction : {e}")
        extraction_results.append({
            'method': method,
            'file': file_path.name,
            'success': False,
            'error': str(e)
        })

    print("\n" + "=" * 60 + "\n")

# Résumé des résultats d'extraction
print("📊 RÉSUMÉ DES EXTRACTIONS :")
print("-" * 30)
for result in extraction_results:
    if result['success']:
        print(f"✅ {result['method'].upper()} - {result['file']}")
        print(f"   ⏱️ {result['extraction_time']:.3f}s | 📏 {result['message_length']} chars | {result['match_status']}")
    else:
        print(f"❌ {result['method'].upper()} - {result['file']}")
        print(f"   ❌ {result['error']}")

print(f"\n🎯 Résultats : {len([r for r in extraction_results if r['success']])}/{len(extraction_results)} extractions réussies")

🔍 Test d'extraction de messages cachés...

🔎 Test d'extraction : Message EXIF
📁 Fichier : test_exif_hidden.jpg
🔧 Méthode : EXIF
--------------------------------------------------
❌ Fichier non trouvé : ..\notebooks\test_outputs\test_exif_hidden.jpg

🔎 Test d'extraction : Message LSB
📁 Fichier : test_lsb_hidden.jpg
🔧 Méthode : LSB
--------------------------------------------------


## ✍️ 5. Test de Signature Stéganographique

Cette section teste les fonctionnalités de signature pour vérifier l'intégrité des messages cachés et détecter d'éventuelles modifications.

In [ ]:
# Test de signature stéganographique
print("✍️ Test de signature stéganographique...\n")

# Message avec signature
signed_message = "Message secret avec signature pour vérifier l'intégrité ! 🔐✍️"
signature_key = "CleSuperSecrete2024!"

print(f"📝 Message à signer : \"{signed_message}\"")
print(f"🔑 Clé de signature : \"{signature_key}\"")
print(f"📏 Longueur : {len(signed_message)} caractères\n")

# Image source pour la signature
source_image = "../test_images/test_image_1.jpg"
signed_output = output_dir / "test_signed_message.jpg"

if os.path.exists(source_image):
    print(f"📷 Image source : {os.path.basename(source_image)}")

    try:
        # Créer un message avec signature
        print(f"\n🔧 Création du message avec signature...")
        sign_result = jpeg_service.sign_hidden_message(
            str(source_image),
            signed_message,
            str(signed_output),
            signature_key,
            method='exif'
        )

        if sign_result['success']:
            print(f"✅ Message signé et caché avec succès !")
            print(f"💾 Fichier créé : {signed_output.name}")
            print(f"🔏 Signature : {sign_result.get('signature', 'N/A')[:20]}...")

            # Vérifier immédiatement la signature
            print(f"\n🔍 Vérification de la signature...")
            verify_result = jpeg_service.verify_signed_message(
                str(signed_output),
                signature_key,
                method='exif'
            )

            if verify_result['success']:
                if verify_result['valid']:
                    print(f"✅ Signature valide ! Message intègre.")
                    print(f"💬 Message vérifié : \"{verify_result['message']}\"")
                    print(f"🏷️ Signature vérifiée : {verify_result.get('signature', 'N/A')[:20]}...")
                else:
                    print(f"❌ Signature invalide ! Message possiblement modifié.")
                    print(f"⚠️ Raison : {verify_result.get('reason', 'Inconnue')}")
            else:
                print(f"❌ Erreur lors de la vérification : {verify_result.get('error', 'Erreur inconnue')}")

        else:
            print(f"❌ Échec de la signature : {sign_result.get('error', 'Erreur inconnue')}")

    except Exception as e:
        print(f"❌ Erreur lors de la signature : {e}")

print("\n" + "=" * 60)

# Test de détection de modification
print("\n🕵️ Test de détection de modification...")

if signed_output.exists():
    try:
        # Simuler une modification en modifiant légèrement l'image
        modified_output = output_dir / "test_modified_signed.jpg"

        print(f"🔧 Simulation d'une modification de l'image...")

        # Ouvrir l'image et la modifier légèrement
        with Image.open(signed_output) as img:
            # Modifier un pixel pour simuler une altération
            pixels = img.load()
            if pixels is not None:
                # Modifier le premier pixel accessible
                try:
                    original_pixel = pixels[0, 0]
                    if isinstance(original_pixel, tuple) and len(original_pixel) >= 3:
                        # Modifier légèrement la composante rouge
                        new_pixel = (
                            min(255, original_pixel[0] + 1),
                            original_pixel[1],
                            original_pixel[2]
                        )
                        if len(original_pixel) > 3:
                            new_pixel = new_pixel + original_pixel[3:]
                        pixels[0, 0] = new_pixel
                        print(f"🎨 Pixel modifié : {original_pixel} → {new_pixel}")
                except:
                    print("⚠️ Impossible de modifier le pixel")

            img.save(modified_output, "JPEG", quality=95)

        print(f"💾 Image modifiée sauvée : {modified_output.name}")

        # Vérifier la signature sur l'image modifiée
        print(f"\n🔍 Vérification de la signature sur l'image modifiée...")
        verify_modified = jpeg_service.verify_signed_message(
            str(modified_output),
            signature_key,
            method='exif'
        )

        if verify_modified['success']:
            if verify_modified['valid']:
                print(f"🤔 Signature encore valide (modification non détectée)")
                print(f"💬 Message : \"{verify_modified['message']}\"")
            else:
                print(f"✅ Modification détectée ! Signature invalide.")
                print(f"⚠️ Raison : {verify_modified.get('reason', 'Modification détectée')}")
        else:
            print(f"❌ Erreur lors de la vérification : {verify_modified.get('error', 'Erreur inconnue')}")

    except Exception as e:
        print(f"❌ Erreur lors du test de modification : {e}")

print("\n" + "=" * 60)

## ⚡ 6. Comparaison des Performances

Cette section compare les performances entre les méthodes EXIF et LSB en termes de vitesse de traitement, capacité et impact sur la taille du fichier.

In [ ]:
# Comparaison des performances
print("⚡ Comparaison des performances EXIF vs LSB...\n")

# Préparer les données de test
test_messages = {
    'court': "Message court 🔒",
    'moyen': "Message de longueur moyenne pour tester les performances de stéganographie JPEG avec différentes méthodes ! " * 3,
    'long': "Message très long pour tester la capacité maximale et les performances. " * 20
}

performance_data = []
source_image = "../test_images/test_image_1.jpg"

if os.path.exists(source_image):
    print(f"📷 Image de test : {os.path.basename(source_image)}")

    # Analyser d'abord la capacité
    try:
        capacity = jpeg_service.analyze_jpeg_capacity(source_image)
        print(f"💾 Capacités disponibles :")
        print(f"   • EXIF : {capacity.get('exif_capacity', 0):,} caractères")
        print(f"   • LSB : {capacity.get('lsb_capacity', 0):,} caractères\n")
    except Exception as e:
        print(f"⚠️ Erreur d'analyse de capacité : {e}\n")

    # Tester chaque combinaison message/méthode
    for msg_type, message in test_messages.items():
        print(f"📝 Test avec message {msg_type} ({len(message)} chars)")
        print("-" * 50)

        for method in ['exif', 'lsb']:
            print(f"🔧 Méthode : {method.upper()}")

            # Vérifier si le message peut être caché
            max_capacity = capacity.get(f'{method}_capacity', 0) if 'capacity' in locals() else float('inf')
            if len(message) > max_capacity:
                print(f"⚠️ Message trop long pour {method.upper()} (max: {max_capacity})")
                continue

            output_file = output_dir / f"perf_{method}_{msg_type}.jpg"

            try:
                # Test de dissimulation
                start_time = time.time()
                hide_result = jpeg_service.hide_message_in_jpeg(
                    source_image, message, str(output_file), method=method
                )
                hide_time = time.time() - start_time

                if hide_result['success']:
                    # Test d'extraction
                    start_time = time.time()
                    extract_result = jpeg_service.extract_message_from_jpeg(
                        str(output_file), method=method
                    )
                    extract_time = time.time() - start_time

                    if extract_result['success']:
                        # Calculer les statistiques
                        original_size = os.path.getsize(source_image)
                        new_size = os.path.getsize(output_file)
                        size_change = new_size - original_size
                        size_change_percent = (size_change / original_size) * 100

                        chars_per_sec_hide = len(message) / hide_time if hide_time > 0 else float('inf')
                        chars_per_sec_extract = len(message) / extract_time if extract_time > 0 else float('inf')

                        # Vérifier l'intégrité
                        integrity_ok = extract_result['message'] == message

                        performance_data.append({
                            'message_type': msg_type,
                            'method': method,
                            'message_length': len(message),
                            'hide_time': hide_time,
                            'extract_time': extract_time,
                            'total_time': hide_time + extract_time,
                            'chars_per_sec_hide': chars_per_sec_hide,
                            'chars_per_sec_extract': chars_per_sec_extract,
                            'size_change': size_change,
                            'size_change_percent': size_change_percent,
                            'integrity_ok': integrity_ok,
                            'success': True
                        })

                        print(f"   ✅ Dissimulation : {hide_time:.3f}s ({chars_per_sec_hide:.0f} chars/s)")
                        print(f"   ✅ Extraction : {extract_time:.3f}s ({chars_per_sec_extract:.0f} chars/s)")
                        print(f"   📁 Taille : {size_change:+,} bytes ({size_change_percent:+.2f}%)")
                        print(f"   🔍 Intégrité : {'✅' if integrity_ok else '❌'}")

                    else:
                        print(f"   ❌ Échec extraction : {extract_result.get('error', 'Erreur inconnue')}")
                else:
                    print(f"   ❌ Échec dissimulation : {hide_result.get('error', 'Erreur inconnue')}")

            except Exception as e:
                print(f"   ❌ Erreur : {e}")

            print()

        print("=" * 60 + "\n")

# Afficher le résumé des performances
if performance_data:
    print("📊 RÉSUMÉ DES PERFORMANCES")
    print("=" * 50)

    # Grouper par méthode
    exif_data = [d for d in performance_data if d['method'] == 'exif' and d['success']]
    lsb_data = [d for d in performance_data if d['method'] == 'lsb' and d['success']]

    print("🔤 Méthode EXIF :")
    if exif_data:
        avg_hide_time = sum(d['hide_time'] for d in exif_data) / len(exif_data)
        avg_extract_time = sum(d['extract_time'] for d in exif_data) / len(exif_data)
        avg_size_change = sum(d['size_change'] for d in exif_data) / len(exif_data)

        print(f"   • Temps dissimulation moyen : {avg_hide_time:.3f}s")
        print(f"   • Temps extraction moyen : {avg_extract_time:.3f}s")
        print(f"   • Changement taille moyen : {avg_size_change:+.0f} bytes")
        print(f"   • Tests réussis : {len(exif_data)}")
    else:
        print("   • Aucune donnée disponible")

    print("\n🔢 Méthode LSB :")
    if lsb_data:
        avg_hide_time = sum(d['hide_time'] for d in lsb_data) / len(lsb_data)
        avg_extract_time = sum(d['extract_time'] for d in lsb_data) / len(lsb_data)
        avg_size_change = sum(d['size_change'] for d in lsb_data) / len(lsb_data)

        print(f"   • Temps dissimulation moyen : {avg_hide_time:.3f}s")
        print(f"   • Temps extraction moyen : {avg_extract_time:.3f}s")
        print(f"   • Changement taille moyen : {avg_size_change:+.0f} bytes")
        print(f"   • Tests réussis : {len(lsb_data)}")
    else:
        print("   • Aucune donnée disponible")

    # Recommandations
    print(f"\n💡 RECOMMANDATIONS :")
    print(f"   • Pour petits messages (< 1KB) : EXIF (plus rapide, moins d'impact)")
    print(f"   • Pour gros messages (> 1KB) : LSB (plus de capacité)")
    print(f"   • Pour intégrité critique : Utiliser la signature stéganographique")

print("\n🎯 Tests de performance terminés !")
print("=" * 60)

## 🎯 7. Conclusion et Nettoyage

Résumé des tests effectués et nettoyage des fichiers temporaires créés pendant les tests.

In [ ]:
# Conclusion et nettoyage
print("🎯 CONCLUSION DES TESTS DE STÉGANOGRAPHIE JPEG")
print("=" * 60)

# Résumé des fonctionnalités testées
features_tested = [
    "📊 Analyse de capacité des images JPEG",
    "🔤 Dissimulation dans les métadonnées EXIF",
    "🔢 Dissimulation avec la méthode LSB",
    "📝 Extraction de messages cachés",
    "✍️ Signature stéganographique",
    "🔍 Vérification d'intégrité",
    "🕵️ Détection de modifications",
    "⚡ Comparaison des performances"
]

print("✅ Fonctionnalités testées avec succès :")
for feature in features_tested:
    print(f"   {feature}")

print(f"\n📊 STATISTIQUES GLOBALES :")
if 'capacity_results' in locals() and capacity_results:
    print(f"   • Images analysées : {len(capacity_results)}")
    avg_exif_capacity = sum(r['exif_capacity'] for r in capacity_results) / len(capacity_results)
    avg_lsb_capacity = sum(r['lsb_capacity'] for r in capacity_results) / len(capacity_results)
    print(f"   • Capacité EXIF moyenne : {avg_exif_capacity:.0f} caractères")
    print(f"   • Capacité LSB moyenne : {avg_lsb_capacity:.0f} caractères")

if 'extraction_results' in locals() and extraction_results:
    successful_extractions = len([r for r in extraction_results if r['success']])
    print(f"   • Extractions réussies : {successful_extractions}/{len(extraction_results)}")

if 'performance_data' in locals() and performance_data:
    successful_tests = len([p for p in performance_data if p['success']])
    print(f"   • Tests de performance : {successful_tests} réussis")

print(f"\n💡 RECOMMANDATIONS D'UTILISATION :")
print(f"   🔤 Méthode EXIF :")
print(f"      • Idéale pour messages courts (< 500 caractères)")
print(f"      • Très rapide (< 0.01s en général)")
print(f"      • Impact minimal sur la taille du fichier")
print(f"      • Parfaite pour métadonnées ou identifiants")

print(f"\n   🔢 Méthode LSB :")
print(f"      • Idéale pour messages longs (> 500 caractères)")
print(f"      • Plus lente mais grande capacité")
print(f"      • Impact variable sur la taille du fichier")
print(f"      • Parfaite pour textes ou données importantes")

print(f"\n   ✍️ Signature stéganographique :")
print(f"      • Essentielle pour vérifier l'intégrité")
print(f"      • Détecte les modifications non autorisées")
print(f"      • Utiliser avec une clé robuste")

# Liste des fichiers créés
print(f"\n📁 FICHIERS CRÉÉS PENDANT LES TESTS :")
if output_dir.exists():
    created_files = list(output_dir.glob("*.jpg"))
    if created_files:
        total_size = 0
        for file in created_files:
            size = file.stat().st_size
            total_size += size
            print(f"   📄 {file.name} ({size:,} bytes)")
        print(f"\n   📊 Total : {len(created_files)} fichiers, {total_size:,} bytes ({total_size/1024:.1f} KB)")
    else:
        print("   (Aucun fichier créé)")

# Option de nettoyage
print(f"\n🧹 NETTOYAGE :")
print(f"Les fichiers de test ont été créés dans : {output_dir}")
print(f"Vous pouvez les supprimer manuellement si nécessaire.")

# Nettoyage optionnel (décommenté si souhaité)
# cleanup = input("Voulez-vous supprimer les fichiers de test ? (y/N): ")
# if cleanup.lower() == 'y':
#     import shutil
#     if output_dir.exists():
#         shutil.rmtree(output_dir)
#         print("✅ Fichiers de test supprimés")
#     else:
#         print("⚠️ Dossier de test introuvable")

print(f"\n🎉 Tests terminés avec succès !")
print(f"⏰ Fin des tests : {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"🔧 Service JPEG Steganography opérationnel et testé !")
print("=" * 60)

## 🖼️ 8. Test Interactif - Sélection d'Image et Signature

Cette section vous permet de sélectionner votre propre image depuis l'explorateur de fichiers, d'y ajouter une signature stéganographique, et de tester les résultats en temps réel.

In [ ]:
# Test interactif - Sélection d'image et signature
print("🖼️ TEST INTERACTIF - SÉLECTION D'IMAGE")
print("=" * 50)

# Importer les outils nécessaires pour la sélection de fichiers
import tkinter as tk
from tkinter import filedialog
import shutil

def select_image_file():
    """Ouvre un dialogue pour sélectionner une image."""
    root = tk.Tk()
    root.withdraw()  # Cacher la fenêtre principale

    # Types de fichiers supportés
    filetypes = [
        ('Images JPEG', '*.jpg *.jpeg'),
        ('Toutes les images', '*.jpg *.jpeg *.png *.bmp *.tiff'),
        ('Tous les fichiers', '*.*')
    ]

    file_path = filedialog.askopenfilename(
        title="Sélectionnez une image JPEG",
        filetypes=filetypes,
        initialdir=os.path.expanduser("~")  # Commencer dans le dossier utilisateur
    )

    root.destroy()
    return file_path

# Interface simple pour la sélection
print("📁 Cliquez sur 'Sélectionner une image' pour choisir votre fichier...")
print("💡 Conseils :")
print("   • Privilégiez les images JPEG pour une meilleure compatibilité")
print("   • Les images plus grandes offrent plus de capacité de dissimulation")
print("   • Les images avec métadonnées EXIF existantes sont idéales")

# Variables pour stocker les informations
selected_image = None
interactive_results = {}

# Fonction d'aide pour afficher les détails d'une image sélectionnée
def analyze_selected_image(image_path):
    """Analyse complète d'une image sélectionnée."""
    if not os.path.exists(image_path):
        return None

    try:
        # Informations de base
        print(f"\n📸 ANALYSE DE L'IMAGE SÉLECTIONNÉE")
        print("-" * 40)
        display_image_info(image_path)

        # Analyse de capacité
        print(f"\n💾 ANALYSE DE CAPACITÉ")
        capacity = jpeg_service.analyze_jpeg_capacity(image_path)

        if 'error' not in capacity:
            print(f"   ✅ Capacité EXIF : {capacity.get('exif_capacity', 0):,} caractères")
            print(f"   ✅ Capacité LSB : {capacity.get('lsb_capacity', 0):,} caractères")

            # Suggestions basées sur la capacité
            exif_cap = capacity.get('exif_capacity', 0)
            lsb_cap = capacity.get('lsb_capacity', 0)

            print(f"\n💡 RECOMMANDATIONS :")
            if exif_cap > 100:
                print(f"   🔤 EXIF recommandé pour messages jusqu'à {exif_cap} caractères")
            if lsb_cap > 1000:
                print(f"   🔢 LSB recommandé pour messages longs (jusqu'à {lsb_cap:,} caractères)")

            return {
                'path': image_path,
                'exif_capacity': exif_cap,
                'lsb_capacity': lsb_cap,
                'file_size': os.path.getsize(image_path)
            }
        else:
            print(f"   ❌ Erreur d'analyse : {capacity['error']}")
            return None

    except Exception as e:
        print(f"❌ Erreur lors de l'analyse : {e}")
        return None

print(f"\n🎯 Prêt pour la sélection interactive !")
print("=" * 50)

In [ ]:
# Interface interactive de sélection et test
print("🎮 INTERFACE INTERACTIVE")
print("=" * 30)

# Étape 1: Sélection de l'image
try:
    print("📂 Ouverture du dialogue de sélection de fichier...")
    selected_image_path = select_image_file()

    if selected_image_path:
        print(f"✅ Image sélectionnée : {os.path.basename(selected_image_path)}")

        # Copier l'image dans notre dossier de travail pour la sécurité
        selected_filename = os.path.basename(selected_image_path)
        safe_filename = f"selected_{int(time.time())}_{selected_filename}"
        working_image = output_dir / safe_filename

        shutil.copy2(selected_image_path, working_image)
        print(f"💾 Copie de travail créée : {working_image.name}")

        # Analyser l'image sélectionnée
        image_analysis = analyze_selected_image(str(working_image))

        if image_analysis:
            print(f"\n🎯 IMAGE PRÊTE POUR LES TESTS !")

            # Étape 2: Configuration du message et de la signature
            print(f"\n📝 CONFIGURATION DU MESSAGE ET SIGNATURE")
            print("-" * 45)

            # Messages prédéfinis ou personnalisés
            predefined_messages = {
                '1': "Message secret court avec signature ! 🔐",
                '2': "Ceci est un message de test plus long pour vérifier la signature stéganographique et l'intégrité des données cachées dans cette image JPEG sélectionnée. 📝🔍",
                '3': f"Image traitée le {datetime.now().strftime('%Y-%m-%d %H:%M:%S')} avec signature automatique. Fichier original: {selected_filename} 📅",
                'custom': "Message personnalisé (vous pouvez le modifier)"
            }

            print("📋 Messages prédéfinis disponibles :")
            for key, msg in predefined_messages.items():
                if key != 'custom':
                    preview = msg[:50] + "..." if len(msg) > 50 else msg
                    print(f"   {key}. {preview}")
                else:
                    print(f"   {key}. {msg}")

            # Configuration automatique basée sur la capacité
            exif_capacity = image_analysis['exif_capacity']
            lsb_capacity = image_analysis['lsb_capacity']

            # Choisir le message par défaut basé sur la capacité
            if exif_capacity >= len(predefined_messages['1']):
                chosen_message = predefined_messages['1']
                recommended_method = 'exif'
                print(f"\n✅ Message par défaut sélectionné (court) : Méthode EXIF recommandée")
            elif lsb_capacity >= len(predefined_messages['1']):
                chosen_message = predefined_messages['1']
                recommended_method = 'lsb'
                print(f"\n✅ Message par défaut sélectionné (court) : Méthode LSB recommandée")
            else:
                chosen_message = "Test 🔒"  # Message minimal
                recommended_method = 'exif'
                print(f"\n⚠️ Capacité limitée : Message minimal sélectionné")

            # Configuration de la signature
            signature_key = f"SecretKey_{int(time.time())}"
            print(f"🔑 Clé de signature générée : {signature_key}")

            print(f"\n📝 Message sélectionné : \"{chosen_message}\"")
            print(f"📏 Longueur : {len(chosen_message)} caractères")
            print(f"🔧 Méthode recommandée : {recommended_method.upper()}")

            # Étape 3: Test de signature et vérification
            print(f"\n🔐 TEST DE SIGNATURE STÉGANOGRAPHIQUE")
            print("-" * 40)

            # Créer le fichier avec signature
            signed_file = output_dir / f"signed_{safe_filename}"

            try:
                print(f"🔧 Création de la signature...")
                sign_start = time.time()

                sign_result = jpeg_service.sign_hidden_message(
                    str(working_image),
                    chosen_message,
                    str(signed_file),
                    signature_key,
                    method=recommended_method
                )

                sign_time = time.time() - sign_start

                if sign_result['success']:
                    print(f"✅ Signature créée avec succès ! ({sign_time:.3f}s)")
                    print(f"💾 Fichier signé : {signed_file.name}")
                    print(f"🔏 Hash signature : {sign_result.get('signature', 'N/A')[:16]}...")

                    # Comparer les tailles
                    original_size = os.path.getsize(working_image)
                    signed_size = os.path.getsize(signed_file)
                    size_diff = signed_size - original_size

                    print(f"📊 Comparaison des tailles :")
                    print(f"   • Original : {original_size:,} bytes")
                    print(f"   • Avec signature : {signed_size:,} bytes")
                    print(f"   • Différence : {size_diff:+,} bytes ({(size_diff/original_size)*100:+.2f}%)")

                    # Vérification immédiate
                    print(f"\n🔍 VÉRIFICATION DE LA SIGNATURE")
                    print("-" * 35)

                    verify_start = time.time()
                    verify_result = jpeg_service.verify_signed_message(
                        str(signed_file),
                        signature_key,
                        method=recommended_method
                    )
                    verify_time = time.time() - verify_start

                    if verify_result['success']:
                        if verify_result['valid']:
                            print(f"✅ SIGNATURE VALIDE ! ({verify_time:.3f}s)")
                            print(f"💬 Message récupéré : \"{verify_result['message']}\"")
                            print(f"🔍 Intégrité : CONFIRMÉE")

                            # Test de résistance - modification simulée
                            print(f"\n🧪 TEST DE RÉSISTANCE AUX MODIFICATIONS")
                            print("-" * 40)

                            # Créer une version modifiée
                            modified_file = output_dir / f"modified_{safe_filename}"

                            try:
                                with Image.open(signed_file) as img:
                                    # Modification subtile de l'image
                                    pixels = img.load()
                                    if pixels and img.size[0] > 10 and img.size[1] > 10:
                                        # Modifier quelques pixels au centre
                                        center_x, center_y = img.size[0] // 2, img.size[1] // 2
                                        original_pixel = pixels[center_x, center_y]

                                        if isinstance(original_pixel, tuple) and len(original_pixel) >= 3:
                                            modified_pixel = (
                                                min(255, max(0, original_pixel[0] + 5)),
                                                original_pixel[1],
                                                original_pixel[2]
                                            )
                                            if len(original_pixel) > 3:
                                                modified_pixel = modified_pixel + original_pixel[3:]
                                            pixels[center_x, center_y] = modified_pixel

                                            print(f"🎨 Pixel ({center_x},{center_y}) modifié : {original_pixel} → {modified_pixel}")

                                    img.save(modified_file, "JPEG", quality=95)

                                # Tester la signature sur l'image modifiée
                                verify_modified = jpeg_service.verify_signed_message(
                                    str(modified_file),
                                    signature_key,
                                    method=recommended_method
                                )

                                if verify_modified['success']:
                                    if verify_modified['valid']:
                                        print(f"🤔 Signature encore valide (modification non critique)")
                                    else:
                                        print(f"✅ MODIFICATION DÉTECTÉE ! Signature invalidée")
                                        print(f"⚠️ Raison : {verify_modified.get('reason', 'Altération détectée')}")
                                else:
                                    print(f"❌ Erreur lors de la vérification : {verify_modified.get('error')}")

                            except Exception as e:
                                print(f"⚠️ Test de modification échoué : {e}")

                            # Résumé final
                            print(f"\n🎉 RÉSUMÉ DU TEST INTERACTIF")
                            print("=" * 35)
                            print(f"📷 Image testée : {selected_filename}")
                            print(f"💾 Taille originale : {original_size:,} bytes")
                            print(f"🔤 Méthode utilisée : {recommended_method.upper()}")
                            print(f"📝 Message : {len(chosen_message)} caractères")
                            print(f"⏱️ Temps signature : {sign_time:.3f}s")
                            print(f"⏱️ Temps vérification : {verify_time:.3f}s")
                            print(f"🔍 Intégrité : ✅ CONFIRMÉE")
                            print(f"🛡️ Résistance : ✅ TESTÉE")

                            # Sauvegarder les résultats
                            interactive_results = {
                                'original_file': selected_filename,
                                'working_file': str(working_image),
                                'signed_file': str(signed_file),
                                'message': chosen_message,
                                'method': recommended_method,
                                'signature_key': signature_key,
                                'sign_time': sign_time,
                                'verify_time': verify_time,
                                'size_original': original_size,
                                'size_signed': signed_size,
                                'integrity_confirmed': True
                            }

                        else:
                            print(f"❌ SIGNATURE INVALIDE !")
                            print(f"⚠️ Raison : {verify_result.get('reason', 'Signature corrompue')}")
                    else:
                        print(f"❌ Erreur de vérification : {verify_result.get('error')}")

                else:
                    print(f"❌ Échec de la signature : {sign_result.get('error')}")

            except Exception as e:
                print(f"❌ Erreur durant le processus de signature : {e}")

        else:
            print(f"❌ Impossible d'analyser l'image sélectionnée")

    else:
        print("⚠️ Aucune image sélectionnée")

except Exception as e:
    print(f"❌ Erreur durant la sélection : {e}")
    print("💡 Assurez-vous que tkinter est installé et disponible")

print(f"\n🏁 Test interactif terminé !")
print("=" * 50)

In [ ]:
# Test complet autonome - Création et extraction immédiate
print("🚀 TEST COMPLET AUTONOME - CRÉATION ET EXTRACTION")
print("=" * 55)

# Trouver une image disponible pour les tests
available_images = [
    "../test_images/test_image_1.jpg",
    "../test_images/test_image_3.jpg",
    "../test_images/test_image_1_steg.jpg",
    "../test_images/test_image_2.png",
    "../test_images/test_image_4.png"
]

source_image = None
for img_path in available_images:
    if os.path.exists(img_path):
        source_image = img_path
        break

if source_image:
    print(f"📷 Image sélectionnée : {os.path.basename(source_image)}")

    # Analyser l'image source
    print(f"\n📊 ANALYSE DE L'IMAGE SOURCE")
    print("-" * 30)
    display_image_info(source_image)

    try:
        capacity = jpeg_service.analyze_jpeg_capacity(source_image)
        print(f"\n💾 Capacités disponibles :")
        print(f"   • EXIF : {capacity.get('exif_capacity', 0):,} caractères")
        print(f"   • LSB : {capacity.get('lsb_capacity', 0):,} caractères")
    except Exception as e:
        print(f"⚠️ Erreur d'analyse de capacité : {e}")

    # Tests pour chaque méthode
    test_scenarios = [
        {
            'method': 'exif',
            'name': 'EXIF',
            'message': "Message test EXIF avec signature automatique ! 🔐📝",
            'output': output_dir / f"auto_test_exif_{int(time.time())}.jpg"
        },
        {
            'method': 'lsb',
            'name': 'LSB',
            'message': "Message test LSB plus long pour démontrer la capacité supérieure de cette méthode. Ce message contient plus de caractères pour tester les performances et l'intégrité. 🔢💾✨",
            'output': output_dir / f"auto_test_lsb_{int(time.time())}.jpg"
        }
    ]

    results_summary = []

    for scenario in test_scenarios:
        method = scenario['method']
        method_name = scenario['name']
        message = scenario['message']
        output_file = scenario['output']

        print(f"\n🔧 TEST {method_name} - DISSIMULATION ET EXTRACTION")
        print("-" * 50)
        print(f"📝 Message : \"{message[:60]}...\"")
        print(f"📏 Longueur : {len(message)} caractères")

        try:
            # Étape 1: Dissimulation
            print(f"\n🔐 Étape 1 : Dissimulation {method_name}...")
            hide_start = time.time()

            hide_result = jpeg_service.hide_message_in_jpeg(
                source_image,
                message,
                str(output_file),
                method=method
            )

            hide_time = time.time() - hide_start

            if hide_result['success']:
                print(f"✅ Dissimulation réussie ! ({hide_time:.3f}s)")
                print(f"💾 Fichier créé : {output_file.name}")

                # Comparer les tailles
                original_size = os.path.getsize(source_image)
                new_size = os.path.getsize(output_file)
                size_diff = new_size - original_size

                print(f"📊 Impact sur la taille :")
                print(f"   • Original : {original_size:,} bytes")
                print(f"   • Nouveau : {new_size:,} bytes")
                print(f"   • Différence : {size_diff:+,} bytes ({(size_diff/original_size)*100:+.2f}%)")

                # Étape 2: Extraction immédiate
                print(f"\n🔍 Étape 2 : Extraction {method_name}...")
                extract_start = time.time()

                extract_result = jpeg_service.extract_message_from_jpeg(
                    str(output_file),
                    method=method
                )

                extract_time = time.time() - extract_start

                if extract_result['success']:
                    extracted_message = extract_result['message']

                    print(f"✅ Extraction réussie ! ({extract_time:.3f}s)")
                    print(f"📏 Message extrait : {len(extracted_message)} caractères")

                    # Vérification d'intégrité
                    if extracted_message == message:
                        integrity_status = "✅ PARFAITE"
                        print(f"🔍 Intégrité : {integrity_status}")
                        print(f"💬 Message vérifié : \"{extracted_message[:50]}...\"")
                    else:
                        integrity_status = "❌ COMPROMISE"
                        print(f"🔍 Intégrité : {integrity_status}")
                        print(f"⚠️ Différence détectée !")
                        print(f"   • Attendu : \"{message[:30]}...\"")
                        print(f"   • Extrait : \"{extracted_message[:30]}...\"")

                    # Statistiques de performance
                    total_time = hide_time + extract_time
                    chars_per_sec = len(message) / total_time if total_time > 0 else float('inf')

                    print(f"\n📈 Statistiques de performance :")
                    print(f"   • Temps total : {total_time:.3f}s")
                    print(f"   • Vitesse : {chars_per_sec:.0f} caractères/seconde")
                    print(f"   • Efficacité : {len(message)/new_size*1000:.2f} chars/KB")

                    # Sauvegarder les résultats
                    results_summary.append({
                        'method': method_name,
                        'message_length': len(message),
                        'hide_time': hide_time,
                        'extract_time': extract_time,
                        'total_time': total_time,
                        'size_original': original_size,
                        'size_new': new_size,
                        'size_diff': size_diff,
                        'integrity': integrity_status,
                        'success': True,
                        'output_file': output_file.name
                    })

                else:
                    print(f"❌ Échec de l'extraction : {extract_result.get('error', 'Erreur inconnue')}")
                    results_summary.append({
                        'method': method_name,
                        'success': False,
                        'error': f"Extraction: {extract_result.get('error', 'Erreur inconnue')}"
                    })

            else:
                print(f"❌ Échec de la dissimulation : {hide_result.get('error', 'Erreur inconnue')}")
                results_summary.append({
                    'method': method_name,
                    'success': False,
                    'error': f"Dissimulation: {hide_result.get('error', 'Erreur inconnue')}"
                })

        except Exception as e:
            print(f"❌ Erreur durant le test {method_name} : {e}")
            results_summary.append({
                'method': method_name,
                'success': False,
                'error': f"Exception: {str(e)}"
            })

        print("=" * 60)

    # Résumé final des résultats
    print(f"\n🎯 RÉSUMÉ FINAL DES TESTS AUTONOMES")
    print("=" * 45)

    successful_tests = [r for r in results_summary if r['success']]
    failed_tests = [r for r in results_summary if not r['success']]

    print(f"✅ Tests réussis : {len(successful_tests)}/{len(results_summary)}")

    if successful_tests:
        print(f"\n📊 Résultats détaillés :")
        for result in successful_tests:
            print(f"\n🔤 Méthode {result['method']} :")
            print(f"   • Message : {result['message_length']} caractères")
            print(f"   • Temps dissimulation : {result['hide_time']:.3f}s")
            print(f"   • Temps extraction : {result['extract_time']:.3f}s")
            print(f"   • Impact taille : {result['size_diff']:+,} bytes")
            print(f"   • Intégrité : {result['integrity']}")
            print(f"   • Fichier créé : {result['output_file']}")

        # Comparaison des méthodes
        if len(successful_tests) > 1:
            print(f"\n⚖️ COMPARAISON DES MÉTHODES :")
            exif_result = next((r for r in successful_tests if r['method'] == 'EXIF'), None)
            lsb_result = next((r for r in successful_tests if r['method'] == 'LSB'), None)

            if exif_result and lsb_result:
                print(f"   • Vitesse : EXIF ({exif_result['total_time']:.3f}s) vs LSB ({lsb_result['total_time']:.3f}s)")
                print(f"   • Impact taille : EXIF ({exif_result['size_diff']:+} bytes) vs LSB ({lsb_result['size_diff']:+} bytes)")

                if exif_result['total_time'] < lsb_result['total_time']:
                    print(f"   🏆 EXIF plus rapide de {((lsb_result['total_time']/exif_result['total_time']-1)*100):.1f}%")
                else:
                    print(f"   🏆 LSB plus rapide de {((exif_result['total_time']/lsb_result['total_time']-1)*100):.1f}%")

    if failed_tests:
        print(f"\n❌ Tests échoués :")
        for result in failed_tests:
            print(f"   • {result['method']} : {result['error']}")

    print(f"\n🎉 Tests autonomes terminés avec succès !")

else:
    print("❌ Aucune image disponible pour les tests !")
    print("💡 Veuillez ajouter des images dans le dossier test_images/")

print("=" * 60)